In [10]:
# !/usr/bin/python
# -*- coding: UTF-8 -*-
# model

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [11]:
class WordVector(nn.Module):
    def __init__(self, vocab_num, embed_dim):
        super().__init__()       
#         word embed
        self.inputword_embed = nn.Embedding(vocab_num, embed_dim, sparse=False)
        self.sample_embed = nn.Embedding(vocab_num, embed_dim, sparse=False)
#         init weight
        initrange = 0.5 / embed_dim
        self.inputword_embed.weight.data.uniform_(-initrange, initrange)
        self.sample_embed.weight.data.uniform_(-initrange, initrange)
        
#         word_ztf: (1, embed_dim + 1) to hidden: (1, embed_dim)
#         self.i2o = nn.Linear(embed_dim + 1, embed_dim)
        
    def forward(self, cen_vocab_id, con_vocabs_id, con_vocabs_weight, neg_samples_id):
#         output: (count input word, embed_dim)
       
#         (count input word, embed_dim, 1)
        input_embed = self.inputword_embed(cen_vocab_id).unsqueeze(2)
        
#         use context matrix adjust word embed
#         (count input word, 2WINDOW, embed_dim)
        con_embed = self.sample_embed(con_vocabs_id)
#         (count input word, 2WINDOW), each row represent an adjusted word embed
        log_con_cen = torch.bmm(con_embed, input_embed).squeeze(2)
# ------weight
        con_vocabs_weight = 1 - F.softmax(con_vocabs_weight, dim=1)
        log_con_cen = log_con_cen.mul(con_vocabs_weight)
        log_con_cen = F.logsigmoid(log_con_cen).sum(1) # sum by col, size: (count input word)
        
# ------sampling types
# and in dataloader.py

#         1. negative
        neg_embed = self.sample_embed(neg_samples_id) # (count input word, 2WINDOW * NEGATIVE, embed_dim)
        log_neg_cen = torch.bmm(neg_embed, -input_embed).squeeze(2) # size: (count input word, 2WINDOW)
        log_neg_cen = F.logsigmoid(log_neg_cen).sum(1) # sum by col, size: (count input word)
        loss = log_con_cen + log_neg_cen
# #         2. no negative
#         loss = log_con_cen
        return (-loss).mean()
    
    def getvectors(self):
        return self.inputword_embed.weight.data